# Drive mount

In [ ]:
#Colab에 파일 업로드 시 직접 업로드보다는 구글 드라이브 연동을 추천합니다.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import

In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments

)
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# 데이터를 Colab 서버 내에 세팅

In [ ]:
# open.zip 파일을 업로드한 경로를 확인하여 적용
!unzip  "/content/drive/MyDrive/open.zip"

Archive:  /content/drive/MyDrive/open.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


# Load Validation / Test dataset

In [ ]:
file_path = '/content/train.csv'
train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True)

val_data=train_data[:10]
train_data = train_data[10:]

val_label_df = val_data[['question', 'answer']]

train_data.head(2)

,id,context,question,answer
10,TRAIN_06504,관광산업 비중이 높은 제주가 지난해 코로나19 사태로 인한 경제 충격을 가장 크게 ...,서비스업 부문에서 서울의 생산이 늘어나게 한 업종은 뭐야,"금융보험, 부동산업"
11,TRAIN_29635,"한국기초과학지원연구원(KBSI, 원장 신형식)이 분석과학 분야 최고 명장을 의미하는...",어떤 명칭을 전자현미경 분석 연구원 이석훈이 얻었어,KBSI 분석과학 마이스터


In [ ]:
# 변환 함수 정의
def transform_row(row):
    context = row['context']
    question = row['question']
    answer = row['answer']
    return f"<bos><start_of_turn>user \n 너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.\n\n Context: {context} Question: {question}<end_of_turn>\n<start_of_turn>model \n {answer} <end_of_turn><eos>"

In [ ]:
train_data['text'] = train_data.apply(transform_row, axis=1)

In [ ]:
train_data = Dataset.from_pandas(train_data[['text']])

In [ ]:
print(train_data[0])

{'text': '<bos><start_of_turn>user \n 너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.\n\n Context: 관광산업 비중이 높은 제주가 지난해 코로나19 사태로 인한 경제 충격을 가장 크게 받은 것으로 나타났다. 반면 서울은 광공업 생산은 줄었지만, 부동산·주식 등 자산 가격 급등으로 서비스업 생산은 오히려 늘어났다.\n22일 통계청이 발표한 ‘2020년 4분기 및 연간 지역경제동향’에 따르면, 지난해 전국 16개 시·도(세종 제외)의 소매판매는 전년 대비 0.2% 줄었다. 관광업 비중이 크고 외국인 관광객이 많이 찾는 제주의 소매판매가 26.9% 쪼그라들어 감소 폭이 가장 컸다. \n제주 외에 서울(-9%), 인천(-8.5%), 부산(-4.9%) 등 면세점이 있는 지역에서 지난해 소비가 급감한 것으로 조사됐다. 반면 전남(4.9%), 경남(1.9%) 등 일부 지역에선 승용차 판매 호조 등에 힘입어 소비가 오히려 늘었다.\n사회적 거리 두기와 직결되는 서비스업 생산은 1년 사이 2% 줄었다. 특히 16개 시·도 가운데 서울을 제외하고 모두 마이너스(-)를 기록했다. 제주(-10.4%), 인천(-9.8%), 강원(-4.9%) 등에서 감소 폭이 컸다. \n반면 서울은 지난해 말 코로나19 3차 확산과 사회적 거리 두기 강화의 직격탄을 맞았지만, 부동산 광풍과 동학개미운동으로 금융보험, 부동산업을 중심으로 서비스업 생산이 늘었다. 통계청 관계자는 “부동산과 금융업은 다른 지역도 어느 정도 늘었지만, 서울에 금융기관 등이 몰려 있어 서울만 서비스업 생산이 늘었다”고 설명했다. \n다만 지난해 제조업 등 광공업 생산은 0.4% 늘었는데, 서울의 경우 1년 사이 14.7%나 급감했다. 의복·모피, 기계장비 등을 중심으로 생산이 줄어든 영향이다. 반면 경기 지역은 광공업 생산이 9%나 늘어나며 모든 시·도 가운데 최대 증가 폭을 기록했다.

# Model Load

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config

)

AttributeError: 'bool' object has no attribute '__code__'

In [ ]:
trainer.train()

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
!ls -alh lora_adapter

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')


In [ ]:
!ls -alh ./gemma-2b-it-sum-ko

In [ ]:
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-sum-ko"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)


# Define F1 score

In [ ]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}


# Validation
- 로드한 모델에 대해 간단한 프롬프트를 입력하여 성능 검증

In [ ]:
qa_pipeline = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(question_prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    response = qa_pipeline(question_prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
    if "Answer:" in response:
            # Answer: 이후에 생성된 토큰 들만을 답변으로 사용합니다.
            response = response.split("Answer:", 1)[1][:20]

            # 토큰 반복 생성 및 노이즈 토큰 관련 처리
            if "Que" in response:
                response = response.split("Que", 1)[0]
            if "⊙" in response:
                response = response.split("⊙", 1)[0]
            if "Con" in response:
                response = response.split("Con", 1)[0]
    return response

In [ ]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
predict_dict = {}
count = 0

for index, row in val_data.iterrows():
    try:
        context = row['context']
        question = row['question']


        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            predict_dict[question] = answer
        else:
            predict_dict[question] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[question])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")



In [ ]:
val_inf_df = pd.DataFrame(list(predict_dict.items()), columns=['question', 'answer'])
val_inf_df.head()

In [ ]:
# f1-score 를 출력합니다.
results = evaluate( val_inf_df, val_label_df)
print(results)

# Inference

In [ ]:
file_path = '/content/test.csv'
test_data = pd.read_csv(file_path)

In [ ]:
# 모델 추론
submission_dict = {}

for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        id = row['id']

        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

    except Exception as e:
        print(f"Error processing question {e}")

# Submission

In [ ]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( '/content/submission.csv', index=False)